# 13. 서브워드 토크나이저(Subword Tokenizer)

## 04) 허깅페이스 토크나이저(Huggingface Tokenizer)

자연어 처리 스타트업 허깅페이스가 개발한 패키지 tokenizers는 자주 등장하는 서브워드들을 하나의 토큰으로 취급하는 다양한 서브워드 토크나이저를 제공합니다. 

이번 실습에서는 이 중에서 WordPiece Tokenizer를 실습해보겠습니다. 실습을 위해 우선 tokenizers를 설치합니다.

```
pip install tokenizers
```

### 1. BERT의 워드피스 토크나이저(BertWordPieceTokenizer)

허깅페이스는 해당 토크나이저를 직접 구현하여 tokenizers라는 패키지를 통해 버트워드피스토크나이저(BertWordPieceTokenizer)를 제공합니다.

여기서는 네이버 영화 리뷰 데이터를 해당 토크나이저에 학습시키고, 이로부터 서브워드의 단어 집합(Vocabulary)을 얻습니다. 그리고 임의의 문장에 대해서 학습된 토크나이저를 사용하여 토큰화를 진행합니다. 우선 네이버 영화 리뷰 데이터를 로드합니다.

```
pip install tokenizers==0.10.1
```

In [5]:
import pandas as pd
import urllib.request
from tokenizers import BertWordPieceTokenizer

ratings.txt라는 파일을 데이터프레임으로 로드한 후, 결측값을 제거하고, 실질적인 리뷰 데이터인 document열에 대해서 naver_review.txt라는 파일로 저장합니다.

In [6]:
naver_df = pd.read_table('datasets/ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('datasets/naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

버트워드피스토크나이저를 설정합니다.

각 인자가 의미하는 바는 다음과 같습니다.

+ lowercase : 대소문자를 구분 여부. True일 경우 구분하지 않음.
+ strip_accents : True일 경우 악센트 제거.

    ex) é → e, ô → o

In [9]:
tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)

네이버 영화 리뷰 데이터를 학습하여 단어 집합을 얻어봅시다.

각 인자가 의미하는 바는 다음과 같습니다.

+ files : 단어 집합을 얻기 위해 학습할 데이터
+ vocab_size : 단어 집합의 크기
+ limit_alphabet : 병합 전의 초기 토큰의 허용 개수.
+ min_frequency : 최소 해당 횟수만큼 등장한 쌍(pair)의 경우에만 병합 대상이 된다.

In [10]:
data_file = 'datasets/naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

tokenizer.train(files=data_file,
                vocab_size=vocab_size,
                limit_alphabet=limit_alphabet,
                min_frequency=min_frequency)

학습이 다 되었다면 vocab을 저장합니다.

In [11]:
# vocab 저장
tokenizer.save_model('datasets/')

['datasets/vocab.txt']

In [12]:
# vocab 로드
df = pd.read_fwf('datasets/vocab.txt', header=None)
df

,0
0,[PAD]
1,[UNK]
2,[CLS]
3,[SEP]
4,[MASK]
...,...
29995,留먮씪�뒗
29996,留먮컰�뿉�
29997,留섏쓣
29998,留쏅룄


실제 토큰화를 수행해봅시다.

In [13]:
encoded = tokenizer.encode('아 배고픈데 짜장면먹고싶다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['아', '배고', '##픈', '##데', '짜장면', '##먹고', '##싶다']
정수 인코딩 : [2111, 20631, 3563, 3362, 24681, 7873, 7378]
디코딩 : 아 배고픈데 짜장면먹고싶다


.ids는 실질적인 딥 러닝 모델의 입력으로 사용되는 정수 인코딩 결과를 출력합니다. 

tokens는 해당 토크나이저가 어떻게 토큰화를 진행했는지를 보여줍니다. 

decode()는 정수 시퀀스를 문자열로 복원합니다.

In [14]:
encoded = tokenizer.encode('커피 한잔의 여유를 즐기다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['커피', '한잔', '##의', '여유', '##를', '즐기', '##다']
정수 인코딩 : [12825, 25647, 3257, 12696, 3379, 10784, 3264]
디코딩 : 커피 한잔의 여유를 즐기다


### 2. 기타 토크나이저

이 외 ByteLevelBPETokenizer, CharBPETokenizer, SentencePieceBPETokenizer 등이 존재하며 선택에 따라서 사용할 수 있습니다.

+ BertWordPieceTokenizer : BERT에서 사용된 워드피스 토크나이저(WordPiece Tokenizer)
+ CharBPETokenizer : 오리지널 BPE
+ ByteLevelBPETokenizer : BPE의 바이트 레벨 버전
+ SentencePieceBPETokenizer : 앞서 본 패키지 센텐스피스(SentencePiece)와 호환되는 BPE 구현체

In [15]:
from tokenizers import ByteLevelBPETokenizer, CharBPETokenizer, SentencePieceBPETokenizer

tokenizer = SentencePieceBPETokenizer()
tokenizer.train('datasets/naver_review.txt', vocab_size=10000, min_frequency=5)

encoded = tokenizer.encode("이 영화는 정말 재미있습니다.")
print(encoded.tokens)

['▁이', '▁영화는', '▁정말', '▁재미있', '습니다.']


In [17]:
tokenizer = CharBPETokenizer()
tokenizer.train('datasets/naver_review.txt', vocab_size=10000, min_frequency=5)

encoded = tokenizer.encode("이 영화는 정말 재미있습니다.")
print(encoded.tokens)

['이</w>', '영화는</w>', '정말</w>', '재미있습니다</w>', '.</w>']
